## Fake News Classifier Using LSTM

### Dataset: https://www.kaggle.com/c/fake-news/data

In [1]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.dropna(inplace = True)
df.shape

(18285, 5)

In [6]:
df.reset_index(inplace = True, drop = True)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Data Preprocessing

In [7]:
# text preprocessing on title of each news
ps = PorterStemmer()
corpus = []
for i in range(len(df)):
    sentence = re.sub('^a-zA-Z', ' ', df['title'][i])
    sentence = sentence.lower()
    words = sentence.split()
    words = [ps.stem(word) for word in words if word not in stopwords.words('english')]
    sentence = ' '.join(words)
    corpus.append(sentence)

In [8]:
X = df.drop(['label'], axis = 1)
y = df['label']

In [9]:
tf.__version__

'2.3.1'

In [10]:
# vocabulary size
voc_size = 5000

### One Hot Representation

In [11]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr[0]

[2064, 2605, 2863, 4946, 622, 1215, 906, 3074, 1772, 1824, 3124]

### Embedding Representation

In [12]:
sent_len = 20
embedded_docs = pad_sequences(one_hot_repr, padding = 'pre', maxlen = sent_len)
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 2064, 2605,
       2863, 4946,  622, 1215,  906, 3074, 1772, 1824, 3124])

In [13]:
# creating model
embedding_vector_features = 40
# creating sequential model object
model = Sequential()
# adding embedding layer
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_len))
# adding LSTM layer with 100 neurons
model.add(LSTM(100))
# adding dense layer
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


### Prepare Data for Model

In [14]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [15]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 0)

### Model Training

In [17]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 8s 40ms/step - loss: 0.3398 - accuracy: 0.8394 - val_loss: 0.1865 - val_accuracy: 0.9234
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1304 - accuracy: 0.9491 - val_loss: 0.1837 - val_accuracy: 0.9254
Epoch 3/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0846 - accuracy: 0.9715 - val_loss: 0.2121 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0554 - accuracy: 0.9825 - val_loss: 0.2593 - val_accuracy: 0.9132
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0313 - accuracy: 0.9900 - val_loss: 0.3141 - val_accuracy: 0.9090
Epoch 6/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0162 - accuracy: 0.9961 - val_loss: 0.3709 - val_accuracy: 0.9099
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.4289 - val_accuracy: 0.9125

### Performance Metrics

In [18]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
matrix = confusion_matrix(y_pred, y_test)
score = accuracy_score(y_pred, y_test)
print(matrix, score)

[[3071  204]
 [ 318 2442]] 0.9135045567522784


### Hyperparameter Tuning Using Dropout Layer

In [19]:
# creating model
embedding_vector_features = 40
# creating sequential model object
model = Sequential()
# adding embedding layer
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_len))
# add a dropout layer
model.add(Dropout(0.3))
# adding LSTM layer with 100 neurons
model.add(LSTM(100))
# add a dropout layer
model.add(Dropout(0.3))
# adding dense layer
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 10s 53ms/step - loss: 0.3302 - accuracy: 0.8416 - val_loss: 0.1910 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 8s 42ms/step - loss: 0.1427 - accuracy: 0.9445 - val_loss: 0.1916 - val_accuracy: 0.9251
Epoch 3/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0951 - accuracy: 0.9665 - val_loss: 0.2057 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0744 - accuracy: 0.9722 - val_loss: 0.2712 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0512 - accuracy: 0.9833 - val_loss: 0.3367 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0375 - accuracy: 0.9872 - val_loss: 0.3290 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0321 - accuracy: 0.9897 - val_loss: 0.4615 - val_accuracy: 0.893

In [21]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
matrix = confusion_matrix(y_pred, y_test)
score = accuracy_score(y_pred, y_test)
print(matrix, score)

[[3101  282]
 [ 288 2364]] 0.9055509527754764
